# Chapter 17: Quantum Singular Value Decomposition

---

**Prerequisites:**
- Python 3.8+
- Qiskit 2.1.2
- See `Chapter02_QuantumSoftware.ipynb` for installation instructions

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit_ibm_runtime import Estimator
from qiskit_aer import Aer
from qiskit.quantum_info import SparsePauliOp
from IPython.display import display
from numpy.polynomial.chebyshev import Chebyshev
from pyqsp.angle_sequence import QuantumSignalProcessingPhases
from Chapter17_QSVT_functions import myQSVT, get_inverse_phases

## Quantum Signal Processing 

In [3]:
# Define polynomial coefficients: P(a) = 2a^2 - 1
# Coefficients in monomial basis: [a^0, a^1, a^2]
pcoefs = [-1, 0, 2]

# Convert from monomial to Chebyshev basis (required by pyqsp)
pcoefs_cheb = np.polynomial.chebyshev.poly2cheb(pcoefs)

# Create Chebyshev polynomial object
poly = Chebyshev(pcoefs_cheb)

# Compute QSP phase angles
phases = QuantumSignalProcessingPhases(poly, signal_operator="Wz")

print("Polynomial: P(a) = 2a² - 1")
print([float(phi) for phi in phases])




Polynomial: P(a) = 2a² - 1
[-0.7903984342278889, 5.10702591327572e-15, 0.7803978925670074]


## QSVT example

In [4]:
# --- Usage Example ---
degree = 5  # You can now set this to any odd integer
kappa = 3    # Based on your eigenvalues 0.25 and 0.75
angles, scale = get_inverse_phases(degree,kappa)

print(f"Generated {len(angles)} angles for degree {degree}")
print(f"Apply a final scale factor of {1/scale} to the output state.")


Chebyshev coefficients (odd terms only): [ 0.          0.44444444  0.         -0.05954426  0.         -0.16267796]
[np.float64(-1.6572589514203349), np.float64(-0.03544644011584275), np.float64(0.23396985344614607), np.float64(0.23396985344614607), np.float64(-0.03544644011584297), np.float64(-0.08646262462543797)]
Generated 6 angles for degree 5
Apply a final scale factor of 3.0 to the output state.


## Inversion

In [5]:
A = np.array([[0.5, -0.25], [-0.25, 0.5]])
b = np.array([1, 0]) # State |0>
degree = 5
kappa = 3
solver = myQSVT(A, b, degree, kappa, nShots=1000)


u_qsvt = solver.execute()

if u_qsvt is not None:
    print("QSVT Solution (|x>):", u_qsvt)
    
    # Classical Verification
    x_exact = np.linalg.solve(A, b)
    u_exact = x_exact / np.linalg.norm(x_exact)
    print("Classical Solution: ", u_exact)
    print("Fidelity:           ", np.abs(np.dot(u_qsvt.conj(), u_exact))**2)

Chebyshev coefficients (odd terms only): [ 0.          0.44444444  0.         -0.05954426  0.         -0.16267796]
[np.float64(-1.6572589514203349), np.float64(-0.03544644011584275), np.float64(0.23396985344614607), np.float64(0.23396985344614607), np.float64(-0.03544644011584297), np.float64(-0.08646262462543797)]
Generated 6 angles for degree 5
Circuit width (total qubits): 4
Circuit depth: 25
QSVT Solution (|x>): [0.87737345 0.47980812]
Classical Solution:  [0.89442719 0.4472136 ]
Fidelity:            0.9986472250550285
